In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier

np.random.seed(0)

In [31]:
df_all_combined = pd.read_csv('df_all_combined.csv')
df_train = pd.read_csv('df_all_combined_train.csv')
df_test = pd.read_csv('df_all_combined_test.csv')
id_test = df_test['id']

<ipython-input-31-b8af859a37fd>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all_combined = pd.read_csv('df_all_combined.csv')


In [32]:
columns_to_drop = ['date_account_created', 'timestamp_first_active', 'id']
for column in columns_to_drop:
    if column in df_train.columns:
        df_train.drop(column, axis=1, inplace=True)

In [33]:
testcolumns_to_drop = ['date_account_created', 'timestamp_first_active', 'country_destination', 'id']
X_test = df_test
for column in testcolumns_to_drop:
    if column in X_test.columns:
        X_test.drop(column, axis=1, inplace=True)

In [34]:
labels = df_train['country_destination'].values
le = LabelEncoder()
y = le.fit_transform(labels) 
X = df_train.drop('country_destination', axis=1, inplace=False)

#Classifier
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X, y)
y_pred = xgb.predict_proba(X_test)  

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

#Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('../sub.csv',index=False)

In [12]:
sub.shape

(310480, 2)

In [6]:
sub.to_csv('../sub.csv',index=False)